In [ ]:
import pandas as pd
import re
from itertools import product
from lightgbm import LGBMClassifier
import pandas as pd
import matplotlib.pylab as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from shapely.geometry import LineString, Point
from geopy.distance import geodesic
import geopandas as gpd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [ ]:

def modelCompare( X_train, y_train, X_test,  y_test):
    # Split the data into train and test sets
    # X_train, X_test, y_train, y_test = TTS(X, y, test_size=0.2, random_state=10)
    
    # Define the classifiers
    classifiers = [
        ("Logistic Regression", LogisticRegression()),
        ("Naive Bayes", GaussianNB()),
        ("Linear Discriminant Analysis", LinearDiscriminantAnalysis()),
        ("K-Nearest Neighbors", KNeighborsClassifier()),
        ("Decision Tree", DecisionTreeClassifier()),
        ("Random Forest", RandomForestClassifier()),
        ("Support Vector Machines", SVC(probability=True)),
        ("Gradient Boosting", GradientBoostingClassifier()),
        ("AdaBoost", AdaBoostClassifier()),
        ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss")),
        ("LightGBM", LGBMClassifier()),
        ("Quadratic Discriminant Analysis", QuadraticDiscriminantAnalysis()),
        ("Gaussian Process Classifier", GaussianProcessClassifier()),
        ("Stochastic Gradient Descent", SGDClassifier()),
        ("Linear Support Vector Machines", LinearSVC()),
        ("Multi-Layer Perceptron", MLPClassifier())
    ]

    # Train and evaluate the classifiers
    results = []
    for name, clf in classifiers:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        result = {
            "Classifier": name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "Confusion Matrix": cm
        }
        results.append(result)

    # Convert the results to a DataFrame and display
    results_df = pd.DataFrame(results)
    results_df = results_df[["Classifier", "Accuracy", "Precision", "Recall", "F1 Score","Confusion Matrix"]]
    return results_df

In [ ]:
def five_fold(X, y):

    X_train,X_test,y_train,y_test = TTS(X,y,test_size=0.3,random_state=420)
    lgbm = LGBMClassifier(n_estimators=80, learning_rate=0.1, max_depth=5, num_leaves=7, min_child_samples=5,
                          reg_alpha=0.0, reg_lambda=0.0, min_split_gain=0.0, subsample=0.5, colsample_bytree=0.5,
                          subsample_freq=3, min_child_weight=0.001)

    kf = KFold(n_splits=5, shuffle=True, random_state=1)

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        lgbm.fit(X_train, y_train)
        y_pred = lgbm.predict(X_test)

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))

    print(f"5-Fold Cross Validation Accuracy: {mean(accuracy_scores)}")
    print(f"5-Fold Cross Validation Precision: {mean(precision_scores)}")
    print(f"5-Fold Cross Validation Recall: {mean(recall_scores)}")
    print(f"5-Fold Cross Validation F1 Score: {mean(f1_scores)}")
    score = pd.DataFrame({'acc': accuracy_scores, 'pre': precision_scores, 'recall':recall_scores, 'f1': f1_scores})
    return lgbm, score

In [ ]:
def getScore(y,y_pred):
    print("Accuracy:{} ".format(accuracy_score(y, y_pred)))
    print("Precision:{} ".format(precision_score(y, y_pred)))
    print("Recall:{} ".format(recall_score(y, y_pred)))
    print("F1 Score:{} ".format(f1_score(y, y_pred)))

In [ ]:
def confusion(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    print("Confusion Matrix:")
    print(cm)
    return cm

In [ ]:
# 三个地市数据进行训练
city_data_train= pd.read_csv('data/city_data_train.csv')
city_data_test= pd.read_csv('data/city_data_test.csv')
X_c_train = city_data_train[['timedelta', 'entype', 'extype', 'cnt', 'oricnt', 'descnt']]
y_c_train = city_data_train[['iscor']]

# 测试集
X_c_test = city_data_test[['timedelta', 'entype', 'extype', 'cnt', 'oricnt', 'descnt']]
y_c_test = city_data_test[['iscor']]

In [ ]:
# 验证集（剩余地市的路网数据）
flag_data = pd.read_csv('data/flag_data.csv')

# 全量数据进行建模预测
X_flag_all = flag_data[['timedelta', 'entype', 'extype', 'cnt', 'oricnt', 'descnt']]
y_flag_all = flag_data[['iscor']]

In [ ]:
flag_c_lgbm, flag_c_score = five_fold(X_c_train, y_c_train)

In [ ]:
getScore(y_c_test, flag_c_lgbm.predict(X_c_test))

In [ ]:
confusion(y_c_test, flag_c_lgbm.predict(X_c_test))

In [ ]:
# 各种算法比较
flag_c_score_allmodel = modelCompare(X_c_train,  y_c_train, X_flag_all, y_flag_all)

In [ ]:
flag_c_score_allmodel

# 区域地市训练的模型预测结果

In [ ]:
getScore(y_flag_all, flag_c_lgbm.predict(X_flag_all))

# 可计算路网绘图
## 门架及收费站经纬度可视化

In [ ]:
flag_node_loc = read_ck("""
select 
    node
    , toFloat64(splitByChar(',', location)[1]) lng
	, toFloat64(splitByChar(',', location)[2]) lat
from (
select
	distinct node 
	, location
from(
select EnNode node,locationEn1 location from topo.dwd_topo dt 
union all
select EnNode node,locationEn1 location  from topo.dwd_topo dt )
where length(location) > 1)""")

In [ ]:
flag_node_loc['nodetype'] = flag_node_loc.node.map(lambda x: 1 if x[-2:] == 'EN' or x[-2:] == 'EX' else 2)

In [ ]:
flag_node_loc['geometry'] = flag_node_loc.apply(lambda x: Point(x.lng, x.lat), axis=1)
flag_node_loc = tools.df2gdf(flag_node_loc)
flag_node_loc.to_file('shp/flag_node_loc.shp', driver='ESRI Shapefile', encoding='utf-8')

# 门架收费站拓扑绘制   

In [ ]:
flag_data_plot = pd.merge(flag_data, flag_node_loc[['node', 'geometry']], left_on='ennode', right_on='node', how='inner')
flag_data_plot = pd.merge(flag_data_plot, flag_node_loc[['node', 'geometry']], left_on='exnode', right_on='node', how='inner')
flag_data_plot['geometry'] = flag_data_plot.apply(lambda x: LineString([x.geometry_x, x.geometry_y]), axis=1)
flag_data_plot = tools.df2gdf(flag_data_plot)
flag_data_plot[['section', 'cnt', 'geometry']]
flag_data_plot[['section', 'cnt', 'geometry']].to_file('shp/ori_flag_topo.shp', driver='ESRI Shapefile', encoding='utf-8')
flag_topo_pred_plot = pd.merge(flag_topo_pred, flag_node_loc[['node', 'geometry']], left_on='ennode', right_on='node', how='inner')
flag_topo_pred_plot = pd.merge(flag_topo_pred_plot, flag_node_loc[['node', 'geometry']], left_on='exnode', right_on='node', how='inner')
flag_topo_pred_plot['geometry'] = flag_topo_pred_plot.apply(lambda x: LineString([x.geometry_x, x.geometry_y]), axis=1)
flag_topo_pred_plot  = tools.df2gdf(flag_topo_pred_plot)
flag_topo_pred_plot[['section', 'cnt', 'geometry']]


# 预测拓扑
flag_topo_pred_plot[['section', 'cnt', 'geometry']].to_file('shp/pred_flag_topo.shp', driver='ESRI Shapefile', encoding='utf-8')

# 正确拓扑
right_flag_topo = read_ck("""
select 
OD,EnNodename ,ExNodename 
, (locationEn[1], locationEn[2]) en
, (locationEx[1], locationEx[2]) ex
from(
select OD,EnNodename ,ExNodename 
, arrayMap(x->toFloat64(x), splitByChar(',', locationEn)) locationEn 
,arrayMap(x->toFloat64(x), splitByChar(',', locationEx)) locationEx 
from topo.dwd_topo_new dtn 
where distance != 0)

""")

right_flag_topo['geometry'] = right_flag_topo.apply(lambda x: LineString([eval(x.en), eval(x.ex)]) , axis=1)
right_flag_topo = tools.df2gdf(right_flag_topo)
right_flag_topo.to_file('shp/right_flag_topo.shp', driver='ESRI Shapefile', encoding='utf-8')

# 门架区域级绘图

In [ ]:
city_data_train['pred'] = flag_c_lgbm.predict(X_c_train)

city_data_train_plot = pd.merge(city_data_train, flag_node_loc[['node', 'geometry']], left_on='ennode', right_on='node', how='inner')
city_data_train_plot = pd.merge(city_data_train_plot, flag_node_loc[['node', 'geometry']], left_on='exnode', right_on='node', how='inner')

city_data_train_plot['geometry'] = city_data_train_plot.apply(lambda x: LineString([x.geometry_x, x.geometry_y]), axis=1)
city_data_train_plot  = tools.df2gdf(city_data_train_plot)
city_data_train_plot[['section', 'cnt', 'geometry', 'iscor', 'pred']]

city_data_train_plot[['section', 'cnt', 'geometry', 'iscor', 'pred']].to_file('shp/city_data_train_plot.shp', driver='ESRI Shapefile', encoding='utf-8')